In [5]:
##지수정보 가져오기  (업데이트)

import win32com.client
import pandas as pd 
import time
import datetime
import numpy as np

column_dailychart = ['code', 'section', 'date', 'time', 'open', 'high', 'low', 'close']

dailychart_prev = pd.read_csv('./dailychart.csv')

instStockChart = win32com.client.Dispatch("CpSysDib.StockChart")
nCpCybos = win32com.client.Dispatch("CpUtil.CpCybos")

row = list(range(len(column_dailychart)))
rows = list()

CPE_MARKET_KIND = {'KOSPI':'U001'}


instStockChart.SetInputValue(1, ord('1'))
instStockChart.SetInputValue(2, (datetime.datetime.today() - datetime.timedelta(days=1)).strftime("%Y%m%d"))
instStockChart.SetInputValue(3, str(np.max(dailychart_prev['date'])+1))
instStockChart.SetInputValue(5, (0, 1, 2, 3, 4, 5))
instStockChart.SetInputValue(6, ord('T'))
instStockChart.SetInputValue(9, ord('1'))

for key, value in CPE_MARKET_KIND.items():

    remain_request_count =  nCpCybos.GetLimitRemainCount(1)
    print(key, value, '남은 요청 : ', remain_request_count)
    
    if remain_request_count == 0:
        print('남은 요청이 모두 소진되었습니다. 잠시 대기합니다.')

        while True:
            time.sleep(2)
            remain_request_count =  nCpCybos.GetLimitRemainCount(1)
            if  remain_request_count > 0:
                print('작업을 재개합니다. (남은 요청 : {0})'.format(remain_request_count))
                break
            print('대기 중...')
            
    instStockChart.SetInputValue(0, value)

    
    # BlockRequest
    instStockChart.BlockRequest()

    # GetHeaderValue
    numData = instStockChart.GetHeaderValue(3)
    numField = instStockChart.GetHeaderValue(1)

    # GetDataValue
    for i in range(numData):
        row[0] =  value
        row[1] = key  # 코스피, 코스닥 여부
        row[2] = instStockChart.GetDataValue(0, i)  # 날짜
        row[3] = instStockChart.GetDataValue(1, i)  # 시간
        row[4] = instStockChart.GetDataValue(2, i)  # 시가
        row[5] = instStockChart.GetDataValue(3, i)  # 고가
        row[6] = instStockChart.GetDataValue(4, i)  # 저가
        row[7] = instStockChart.GetDataValue(5, i)  # 종가
        rows.append(list(row))  
    
    
print('데이터를 모두 불러왔습니다.')

dailychart= pd.DataFrame(data = rows, columns= column_dailychart)
dailychart = dailychart.append(dailychart_prev)
dailychart = dailychart.sort_values(by=['code','date'])
dailychart.to_csv('dailychart_index.csv', index=False)

print('모든 데이터를 저장하였습니다.')

KOSPI U001 남은 요청 :  60
데이터를 모두 불러왔습니다.
모든 데이터를 저장하였습니다.


In [48]:
import urllib.request
from bs4 import BeautifulSoup
import json
from urllib import parse
from collections import OrderedDict
from datetime import datetime


basic_url = "https://finance.naver.com/sise/"

import pandas as pd
import time

# 빈 데이터프레임 생성
df = pd.DataFrame({'Time': [0], 'Result': [0]})
while True:
    # 현재 시간을 기록
    current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())
    
    fp = urllib.request.urlopen(basic_url)
    source = fp.read()
    fp.close()

    soup = BeautifulSoup(source, 'html.parser')
    soup = soup.findAll("span",class_="num")
    kospi_value = soup[1].string #코스피 지수
    # 결과값을 변수에 저장
    
    # 결과값을 데이터프레임에 추가
    
    df = df.append({'Time': [current_time], 'Result': [kospi_value]}, ignore_index=True)
    # 10초 대기
    
    time.sleep(5)
    df

KeyboardInterrupt: 

In [51]:
from bs4 import BeautifulSoup
import urllib.request as req
import re

url = "https://finance.naver.com/marketindex/"

res = req.urlopen(url)
soup = BeautifulSoup(res,"html.parser", from_encoding='euc-kr')

name_nation = soup.select('exchangeList > li.on > a.head.usd > div')
name_price = soup.select('span.value')

#exchangeList > li.on > a.head.usd > div > span.value
price=name_price[0]
price

<span class="value">1,295.00</span>

In [45]:
import pandas as pd
base_url = 'https://m.stock.naver.com/api/json/sise/dailySiseIndexListJson.nhn?code=KOSPI&pageSize=100&page={}'

url = base_url.format(1) #첫페이지만 가지고 온다.

res = requests.get(url)
if res.status_code == 200:
    data = res.json()
    kospi_list = data['result']['siseList']
    print(kospi_list)
    df = pd.DataFrame(kospi_list)
df

[{'cd': 'KOSPI', 'dt': '20230711', 'ncv': 2562.49, 'cv': 41.79, 'cr': 1.66, 'ov': 2538.84, 'hv': 2562.49, 'lv': 2534.48}, {'cd': 'KOSPI', 'dt': '20230710', 'ncv': 2520.7, 'cv': -6.01, 'cr': -0.24, 'ov': 2528.08, 'hv': 2539.63, 'lv': 2517.67}, {'cd': 'KOSPI', 'dt': '20230707', 'ncv': 2526.71, 'cv': -29.58, 'cr': -1.16, 'ov': 2547.41, 'hv': 2547.41, 'lv': 2515.07}, {'cd': 'KOSPI', 'dt': '20230706', 'ncv': 2556.29, 'cv': -22.71, 'cr': -0.88, 'ov': 2574.75, 'hv': 2575.45, 'lv': 2550.67}, {'cd': 'KOSPI', 'dt': '20230705', 'ncv': 2579.0, 'cv': -14.31, 'cr': -0.55, 'ov': 2593.47, 'hv': 2601.99, 'lv': 2578.66}, {'cd': 'KOSPI', 'dt': '20230704', 'ncv': 2593.31, 'cv': -9.16, 'cr': -0.35, 'ov': 2604.72, 'hv': 2607.66, 'lv': 2591.86}, {'cd': 'KOSPI', 'dt': '20230703', 'ncv': 2602.47, 'cv': 38.19, 'cr': 1.49, 'ov': 2580.89, 'hv': 2604.63, 'lv': 2580.49}, {'cd': 'KOSPI', 'dt': '20230630', 'ncv': 2564.28, 'cv': 14.26, 'cr': 0.56, 'ov': 2558.43, 'hv': 2571.03, 'lv': 2541.72}, {'cd': 'KOSPI', 'dt': '20

,cd,dt,ncv,cv,cr,ov,hv,lv
0,KOSPI,20230711,2562.49,41.79,1.66,2538.84,2562.49,2534.48
1,KOSPI,20230710,2520.70,-6.01,-0.24,2528.08,2539.63,2517.67
2,KOSPI,20230707,2526.71,-29.58,-1.16,2547.41,2547.41,2515.07
3,KOSPI,20230706,2556.29,-22.71,-0.88,2574.75,2575.45,2550.67
4,KOSPI,20230705,2579.00,-14.31,-0.55,2593.47,2601.99,2578.66
...,...,...,...,...,...,...,...,...
95,KOSPI,20230221,2458.96,3.84,0.16,2457.51,2466.07,2446.00
96,KOSPI,20230220,2455.12,3.91,0.16,2453.25,2463.17,2430.27
97,KOSPI,20230217,2451.21,-24.27,-0.98,2447.66,2471.51,2445.74
98,KOSPI,20230216,2475.48,47.58,1.96,2444.06,2475.99,2442.07


In [ ]:
def extract_numbers(text):
    # 정규식 패턴을 사용하여 숫자만 추출
    pattern = r'\d+'
    numbers = re.findall(pattern, text)
    return numbers
    